### Datensätze zur Auswahl:
#### Bike Sharing Nutzung:
Der Datensatz dient dazu, die benötigte Anzahl an Mietfahrrädern pro Stunde vorherzusagen, um eine stabile Versorgung sicherzustellen. Ziel ist es, Wartezeiten zu minimieren und die Verfügbarkeit der Fahrräder in Städten zu optimieren. Dafür enthält der Datensatz: Wetterdaten wie Temperatur, Luftfeuchtigkeit, Windgeschwindigkeit, Sichtweite, Taupunkt, Sonneneinstrahlung, Schneefall und Regen, die stündliche Anzahl der ausgeliehenen Fahrräder sowie Datumsangaben.
#### Energie Effizienz von Gebäuden:
Der Datensatz umfasst 768 simulierte Gebäudemodelle, die sich hinsichtlich Form, Verglasungsfläche, deren Verteilung, Ausrichtung und weiteren Parametern unterscheiden. Für jede Probe stehen 8 Merkmale zur Verfügung, die dazu dienen, zwei kontinuierliche Zielwerte vorherzusagen. Alternativ kann der Datensatz auch für Klassifikationsprobleme genutzt werden, indem die Zielwerte auf ganze Zahlen gerundet werden. Der Zweck der Datenerhebung liegt in der Analyse des Energieverbrauchs basierend auf den Designparametern der Gebäude, um Erkenntnisse für energieeffizientes Bauen zu gewinnen.
#### Wein Qualität
Leon macht das....

### Auswahl
Wir haben uns für den Datensatz "Bike Sharing Nutzung" entschieden

## Beschreibung des Datensatzes:


### Package and Data Imports 

In [1]:
import numpy as np
import matplotlib
import pandas as pd

# Define the mappings
season_mapping = {
    "Spring": 1,
    "Summer": 2,
    "Fall": 3,
    "Winter": 4
}
binary_mapping = {
    "Yes": 1,
    "No": 0
}
holiday_mapping = {
    "No Holiday": 0,
    "Holiday": 1
}

# Use the converters parameter to map both columns
df = pd.read_csv('data/SeoulBikeData.csv', 
    converters={
        'Season': lambda x: season_mapping[x], 
        'Holiday': lambda x: holiday_mapping[x],
        'Functioning Day': lambda x: binary_mapping[x]
    }
)

print(df.shape)

(8760, 14)


### Data Exploration
Anzahl Instanzen: 8760
Anzahl Variablen: 14
Meaningfull names: No clarification needed.

In [2]:
df.describe()

,Rented Bike Count,Hour,Temperature(C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000
mean,704.602055,11.500000,12.882922,58.226256,1.724909,1436.825799,4.073813,0.569111,0.148687,0.075068,0.049315,0.966324
std,644.997468,6.922582,11.944825,20.362413,1.036300,608.298712,13.060369,0.868746,1.128193,0.436746,0.216537,0.180404
min,0.000000,0.000000,-17.800000,0.000000,0.000000,27.000000,-30.600000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,191.000000,5.750000,3.500000,42.000000,0.900000,940.000000,-4.700000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,504.500000,11.500000,13.700000,57.000000,1.500000,1698.000000,5.100000,0.010000,0.000000,0.000000,0.000000,1.000000
75%,1065.250000,17.250000,22.500000,74.000000,2.300000,2000.000000,14.800000,0.930000,0.000000,0.000000,0.000000,1.000000
max,3556.000000,23.000000,39.400000,98.000000,7.400000,2000.000000,27.200000,3.520000,35.000000,8.800000,1.000000,1.000000


In [3]:
strong_threshold = 0.7
mediocre_threshold = 0.4

numerical_df = df.select_dtypes(include=['float64', 'int64'])
correlation_matrix = numerical_df.corr()

strong_correlations = correlation_matrix[(correlation_matrix >= strong_threshold) | (correlation_matrix <= -strong_threshold)]

mediocre_correlations = correlation_matrix[
    ((correlation_matrix >= mediocre_threshold) & (correlation_matrix < strong_threshold)) |
    ((correlation_matrix <= -mediocre_threshold) & (correlation_matrix > -strong_threshold))
]

# remove self-correlations (diagonal of ones)
filtered_strong = strong_correlations.where(~(correlation_matrix == 1))
filtered_mediocre = mediocre_correlations.where(~(correlation_matrix == 1))
print("\nFiltered Strong Correlations (without self-correlations):")
print(filtered_strong)

print("\nFiltered Mediocre Correlations (without self-correlations):")
print(filtered_mediocre)


Filtered Strong Correlations (without self-correlations):
                          Rented Bike Count  Hour  Temperature(C)  \
Rented Bike Count                       NaN   NaN             NaN   
Hour                                    NaN   NaN             NaN   
Temperature(C)                          NaN   NaN             NaN   
Humidity(%)                             NaN   NaN             NaN   
Wind speed (m/s)                        NaN   NaN             NaN   
Visibility (10m)                        NaN   NaN             NaN   
Dew point temperature(C)                NaN   NaN        0.912798   
Solar Radiation (MJ/m2)                 NaN   NaN             NaN   
Rainfall(mm)                            NaN   NaN             NaN   
Snowfall (cm)                           NaN   NaN             NaN   
Holiday                                 NaN   NaN             NaN   
Functioning Day                         NaN   NaN             NaN   

                          Humidity(%)  Wind

### Ergebnis:
#### Starke Korrelation:
- Dew point temperature und Temperature korrelieren sehr stark mit einem korrelationsfaktor von 0.91. Dies ist auch einleuchtend.
#### Mittelmäßige Korrelation
- Hour und Rented Bike Count mit 0.41
- Temperatur und Rented Bike Count mit 0.54
- Visibility und Humidity mit -0.54
- Dew point temperature und Humidity mit -0.54
- Solar Radiation und Humidity mit -0.46
 